In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy.spatial import distance

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv("/kaggle/input/fifa-20-complete-player-dataset/players_20.csv")

#### This data frame has more than 100 columns so lets choose the best and useful columns so that we have a good pred 

#### Dividing df into 5 parts to check each column an then concatenate the useful columns 

In [ ]:
df1=df.iloc[:,0:20]
df2=df.iloc[:,20:40]
df3=df.iloc[:,40:60]
df4=df.iloc[:,60:80]
df5=df.iloc[:,80:]

In [ ]:
df1.info()

In [ ]:
df1.describe().transpose()

In [ ]:
df1.columns

In [ ]:
df1=df1.drop(['sofifa_id', 'player_url', 'long_name', 'age', 'dob',
       'height_cm', 'weight_kg', 'nationality', 'club', 'overall', 'potential',
       'value_eur', 'wage_eur','international_reputation','player_positions','preferred_foot'],axis=1)

#### I feel none of the above columns are good to compare two players on

In [ ]:
df1.head()

In [ ]:
df2.head()

In [ ]:
df2.columns

In [ ]:
df2=df2.drop(['body_type', 'real_face', 'release_clause_eur', 'team_jersey_number', 'loaned_from', 'joined',
       'contract_valid_until', 'nation_jersey_number',"nation_position",'player_tags','team_position','gk_diving', 'gk_handling', 'gk_kicking'],axis=1)

In [ ]:
df2.head()

In [ ]:
df2.info()

In [ ]:
df2=df2.fillna(0)

In [ ]:
df2.info()

In [ ]:
df3.info()

In [ ]:
df3.columns

In [ ]:
df3=df3.drop(["player_traits",'gk_reflexes', 'gk_speed', 'gk_positioning'],axis=1)

In [ ]:
df3.head()

In [ ]:
df3=df3.fillna(0)

In [ ]:
df4.info()

In [ ]:
df4.head()

In [ ]:
df4=df4.drop(["ls","st"],axis=1)

In [ ]:
df1.columns

In [ ]:
df2.columns

In [ ]:
df3.columns

In [ ]:
df4.columns

In [ ]:
df=pd.concat([df1,df2,df3,df4],axis=1)

In [ ]:
df.head()

In [ ]:
df["weak_foot"].unique()

In [ ]:
df["skill_moves"].unique()

#### Now splitting the work rate column into defensive and offensive work rate 

In [ ]:
def wr(x):
    x=x.split("/")
    return x[0]
df["offensive_work_rate"]=df["work_rate"].apply(lambda x: wr(x))

In [ ]:
def wr(x):
    x=x.split("/")
    return x[1]
df["defensive_work_rate"]=df["work_rate"].apply(lambda x: wr(x))

In [ ]:
df=df.drop(["work_rate"],axis=1)

In [ ]:
df.head()

In [ ]:
df["offensive_work_rate"].unique()

In [ ]:
def wr(x):
    if x=="High":
        return 0
    elif x=="Medium":
        return 1
    else:
        return 2
df["offensive_work_rate"]=df["offensive_work_rate"].apply(lambda x: wr(x))

In [ ]:
def wr(x):
    if x=="High":
        return 0
    elif x=="Medium":
        return 1
    else:
        return 2
df["defensive_work_rate"]=df["defensive_work_rate"].apply(lambda x: wr(x))

In [ ]:
df.head()

### our final data frame 

In [ ]:
df.info()

In [ ]:
X=df.drop(["short_name"],axis=1)

In [ ]:
X

In [ ]:
player_name="L. Messi"
p_ind=df[df["short_name"]==player_name].index[0]

#### Finding the index of the player 

In [ ]:
dist=[]
for i in range (0,len(X.index)):
    dist.append(distance.euclidean(X.iloc[p_ind].values,X.iloc[i].values))

#### Calculating the euclidean distance of messi wrt to other players and appending all values in a list 

In [ ]:
len(dist)

In [ ]:
pd.Series(dist)

In [ ]:
sn=df["short_name"].to_list()

In [ ]:
sim={"name":sn,"distance":dist}

In [ ]:
sim=pd.DataFrame(sim)

#### Creating a data frame with shortname and the distances 

In [ ]:
sim.iloc[sim["distance"].sort_values().index].head(10)

Sorting down the values wrt to distance

Now lets use cosine similarity and see what the results would be 

In [ ]:
cossim=[]
for i in range (0,len(X.index)):
    cossim.append(1 - spatial.distance.cosine(X.iloc[p_ind].values,X.iloc[i].values))
pd.Series(cossim)
sim2={"name":sn,"cossim":cossim}
sim2=pd.DataFrame(sim2)

In [ ]:
sim2.iloc[sim2["cossim"].sort_values(ascending=False).index].head(10)

In [ ]:
sim.iloc[sim["distance"].sort_values().index].head(10)

#### comparing both the metrics the cossine similarity looks better because
### If we use Euclidean people who have higher attributes will always pop up near messi 
#### But if we use cosine similarity we can really find out the true similar players even if they are lower rated

### using standard scaler to improve the model a bit

In [ ]:
X

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
std=StandardScaler()

In [ ]:
X=std.fit_transform(X)

In [ ]:
cossim=[]
for i in range (0,len(X)):
    cossim.append(1 - spatial.distance.cosine(X[p_ind],X[i]))
pd.Series(cossim)
sim2={"name":sn,"cossim":cossim}
sim2=pd.DataFrame(sim2)

In [ ]:
sim2.iloc[sim2["cossim"].sort_values(ascending=False).index].head(10)

In [ ]:
dist=[]
for i in range (0,len(X)):
    dist.append(distance.euclidean(X[p_ind],X[i]))
pd.Series(dist)
sim={"name":sn,"distance":dist}
sim=pd.DataFrame(sim)
sim.iloc[sim["distance"].sort_values().index].head(10)

#### still Dunno who this slidiero guy is but the scaled model says he is the closest to messi 
#### while the unscaled model says neymar is the closest to messi 